In [2]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
import os
import sys

import clip

import importlib

import contextlib

import json
import os
import random


sys.path.insert(0, os.path.abspath('../..')) 
sys.path.insert(0, os.path.abspath('../../..'))
sys.path.insert(0, os.path.abspath('../../../../stablediffusion'))

In [3]:

import numpy as np
import torch 
from collections import defaultdict
from utils import get_default_path
from utils import Stopwatch
import time
import datetime

In [4]:
import ai.stabledisco as sd
import ai.torchmodules as torchmodules
import ai.torchmodules.data as torchdata
import ai.torchmodules.utils as torchutils
import ai.stabledisco.utils as sdutils
import ai.stabledisco.data as sddata
import ai.stabledisco.constants as sdconsts
import ai.torchmodules.pruning as torchprune
import torch
import torch.nn as nn
import pandas as pd

import ai.stabledisco.decoderpipeline as decoderpipeline
  


torch.multiprocessing.set_start_method('spawn')

In [6]:
tmp_dir =  "/home/ubuntu/tmp/"
tokens_path = os.path.join(tmp_dir, "text_tokens.pk")
rev_tokens_path = os.path.join(tmp_dir, "rev_text_tokens.pk")

df_path = os.path.join("/home/ubuntu/Main/tmp/", "laion_features.feather")
rating_features_path = os.path.join(tmp_dir, "laion_features.pk")
features_path = os.path.join(tmp_dir, "text_features_curr.pk")
tokens_path = os.path.join(tmp_dir, "text_tokens_curr.pk")
rev_tokens_path = os.path.join(tmp_dir, "rev_text_tokens_curr.pk")

features_path_old = os.path.join(tmp_dir, "text_features_old.pk")
tokens_path_old = os.path.join(tmp_dir, "text_tokens_old.pk")
rev_tokens_path_old = os.path.join(tmp_dir, "rev_text_tokens_old.pk")

#prompt_dataframe = pd.read_feather(df_path)

curr_toks = torch.load(tokens_path)
old_toks = torch.load(tokens_path_old)

In [ ]:
prompt_dataframe = pd.read_feather(df_path)
rating_tokens = np.array([np.array(toks) for toks in prompt_dataframe["text_tokens"]])
rating_tokens = torch.tensor(rating_tokens)
del prompt_dataframe

In [ ]:
del curr_toks
del old_toks

In [ ]:

rating_features_path = os.path.join(tmp_dir, "laion_features.pk")
ratings_path = os.path.join(tmp_dir, "laion_ratings.pk")

In [18]:
del prompt_dataframe

torch.save(ratings, ratings_path)
del ratings

In [4]:
if len(text_tokens.shape) > 1:
    text_tokens.view(1, text_tokens.size(0))
flipped = torch.flip(text_tokens, dims=(1,))
end_idx_arg = torch.argwhere(flipped == sdconsts.eot_token)[:,1]
for idx in range(flipped.size(0)):
    flipped[idx] = torch.cat((flipped[idx,end_idx_arg[idx]:], flipped[idx,:end_idx_arg[idx]]))


text_tokens = torch.save(flipped, rev_tokens_path)

In [7]:
from ldm.modules.encoders.modules import FrozenOpenCLIPEmbedder
vit14_clip_model = FrozenOpenCLIPEmbedder()
vit14_clip_model = vit14_clip_model.cuda()


In [8]:
import open_clip
from ldm.modules.encoders.modules import FrozenOpenCLIPEmbedder
vit14_clip_model = FrozenOpenCLIPEmbedder()
vit14_clip_model = vit14_clip_model.cuda()

_, _, vit14_clip_preprocess = open_clip.create_model_and_transforms('ViT-H-14')
clip_model = sd.ClipModel(vit14_clip_model.float(), vit14_clip_preprocess, "ViT-H-14").float()

In [5]:
from ldm.modules.encoders.modules import FrozenOpenCLIPEmbedder
vit14_clip_model, _,  vit14_clip_preprocess = open_clip.create_model_and_transforms('ViT-H-14', device=torch.device('cuda'), pretrained='laion2b_s32b_b79k')
vit14_clip_model = vit14_clip_model.cuda()

vit14_clip_model = vit14_clip_model.eval()
for param in vit14_clip_model.parameters():
    param.requires_grad = False
    
clip_model = sd.ClipModel(vit14_clip_model, vit14_clip_preprocess, "ViT-H-14").float()

In [36]:
try:
    del prev_vals
except:
    pass
try:
    del new_features
except:
    pass
torchutils.torch_garbage_collect()
feature_stack = []
step = 12000
clip_model = clip_model.half()
clip_model._model = clip_model._model.half()
prev_vals = None
cuda_device = torchutils.get_default_device()

token_loader = torch.utils.data.DataLoader(
        curr_toks, num_workers=16, batch_size=step, shuffle=False, pin_memory=True)

start = time.perf_counter()
cnt_finished = 0
with torch.no_grad():
    for batch in token_loader:
        new_features = clip_model.features_from_tokens(batch.to(cuda_device, non_blocking=True), step_size=step)
        if prev_vals is not None:
            feature_stack.append(prev_vals.cpu())
        prev_vals = new_features
        del new_features
        
        cnt_finished += step
        since_start = time.perf_counter() - start
        toks_per_sec = cnt_finished / since_start
        rem_secs = (len(curr_toks) - cnt_finished) / toks_per_sec
        print(f"encode/sec {toks_per_sec:0.2f}")
        print(f"Rem minutes {rem_secs// 60}")
        
    feature_stack.append(prev_vals.cpu())

    features = torch.cat(feature_stack, axis=0)
    del curr_toks
    del feature_stack
torch.save(features, features_path)

encode/sec 1239.71
Rem minutes 134.0
encode/sec 1490.05
Rem minutes 111.0
encode/sec 1596.82
Rem minutes 103.0
encode/sec 1655.15
Rem minutes 100.0
encode/sec 1691.35
Rem minutes 97.0
encode/sec 1715.54
Rem minutes 96.0
encode/sec 1734.01
Rem minutes 95.0
encode/sec 1747.65
Rem minutes 94.0
encode/sec 1758.28
Rem minutes 93.0
encode/sec 1763.57
Rem minutes 93.0
encode/sec 1770.52
Rem minutes 92.0
encode/sec 1776.29
Rem minutes 92.0
encode/sec 1781.05
Rem minutes 92.0
encode/sec 1785.02
Rem minutes 91.0
encode/sec 1788.29
Rem minutes 91.0
encode/sec 1791.29
Rem minutes 91.0
encode/sec 1793.88
Rem minutes 91.0
encode/sec 1796.14
Rem minutes 90.0
encode/sec 1798.16
Rem minutes 90.0
encode/sec 1798.44
Rem minutes 90.0
encode/sec 1800.19
Rem minutes 90.0
encode/sec 1801.65
Rem minutes 90.0
encode/sec 1802.96
Rem minutes 89.0
encode/sec 1804.19
Rem minutes 89.0
encode/sec 1805.31
Rem minutes 89.0
encode/sec 1806.34
Rem minutes 89.0
encode/sec 1807.28
Rem minutes 89.0
encode/sec 1808.12
Rem m

In [38]:
try:
    del prev_vals
except:
    pass
try:
    del new_features
except:
    pass
torchutils.torch_garbage_collect()
feature_stack = []
step = 12000
clip_model._model = clip_model._model.half()
clip_model = clip_model.half()
prev_vals = None
cuda_device = torchutils.get_default_device()

token_loader = torch.utils.data.DataLoader(
        old_toks, num_workers=16, batch_size=step, shuffle=False, pin_memory=True)

start = time.perf_counter()
cnt_finished = 0
with torch.no_grad():
    for batch in token_loader:
        new_features = clip_model.features_from_tokens(batch.to(cuda_device, non_blocking=True), step_size=step)
        if prev_vals is not None:
            feature_stack.append(prev_vals.cpu())
        prev_vals = new_features
        del new_features
        
        cnt_finished += step
        since_start = time.perf_counter() - start
        toks_per_sec = cnt_finished / since_start
        rem_secs = (len(old_toks) - cnt_finished) / toks_per_sec
        print(f"encode/sec {toks_per_sec:0.2f}")
        print(f"Rem minutes {rem_secs// 60}")
        
    feature_stack.append(prev_vals.cpu())

    features = torch.cat(feature_stack, axis=0)
    del old_toks
    del feature_stack
torch.save(features, features_path_old)

encode/sec 862.83
Rem minutes 355.0
encode/sec 1185.56
Rem minutes 258.0
encode/sec 1353.35
Rem minutes 226.0
encode/sec 1455.26
Rem minutes 210.0
encode/sec 1524.02
Rem minutes 200.0
encode/sec 1572.97
Rem minutes 194.0
encode/sec 1609.03
Rem minutes 190.0
encode/sec 1636.80
Rem minutes 186.0
encode/sec 1658.75
Rem minutes 184.0
encode/sec 1673.41
Rem minutes 182.0
encode/sec 1687.94
Rem minutes 180.0
encode/sec 1700.06
Rem minutes 179.0
encode/sec 1710.31
Rem minutes 178.0
encode/sec 1719.04
Rem minutes 177.0
encode/sec 1726.57
Rem minutes 176.0
encode/sec 1733.02
Rem minutes 175.0
encode/sec 1738.96
Rem minutes 174.0
encode/sec 1744.28
Rem minutes 174.0
encode/sec 1748.98
Rem minutes 173.0
encode/sec 1751.87
Rem minutes 173.0
encode/sec 1755.78
Rem minutes 172.0
encode/sec 1759.38
Rem minutes 172.0
encode/sec 1762.57
Rem minutes 171.0
encode/sec 1765.51
Rem minutes 171.0
encode/sec 1768.24
Rem minutes 170.0
encode/sec 1770.74
Rem minutes 170.0
encode/sec 1773.00
Rem minutes 170.0
en

In [39]:
try:
    del prev_vals
except:
    pass
try:
    del new_features
except:
    pass
torchutils.torch_garbage_collect()
feature_stack = []
step = 12000
clip_model._model = clip_model._model.half()
clip_model = clip_model.half()
prev_vals = None
cuda_device = torchutils.get_default_device()

token_loader = torch.utils.data.DataLoader(
        rating_tokens, num_workers=16, batch_size=step, shuffle=False, pin_memory=True)

start = time.perf_counter()
cnt_finished = 0
prev_batch = None
with torch.no_grad():
    for batch in token_loader:
        curr_batch = batch.to(cuda_device, non_blocking=True)
        if prev_batch is not None:
            feature_stack.append(clip_model.features_from_tokens(prev_batch, step_size=step).cpu())
        prev_batch = curr_batch
        
        cnt_finished += step
        since_start = time.perf_counter() - start
        toks_per_sec = cnt_finished / since_start
        rem_secs = (len(rating_tokens) - cnt_finished) / toks_per_sec
        print(f"encode/sec {toks_per_sec:0.2f}")
        print(f"Rem minutes {rem_secs// 60}")
        
    feature_stack.append(clip_model.features_from_tokens(prev_batch, step_size=step).cpu())

    features = torch.cat(feature_stack, axis=0)
    del feature_stack
    del rating_tokens
torch.save(features, rating_features_path)

NameError: name 'rating_tokens' is not defined

In [15]:
def test():
    global clip_model
    tokens = open_clip.tokenize("This is a painting features a lovely young woman strolling through a forest at night.")
    clip_model = clip_model.half()
    half_feats = clip_model.features_from_tokens(tokens.to(cuda_device, non_blocking=True), step_size=step)
    tokens = open_clip.tokenize("This is a painting features a lovely young woman strolling through a forest at night.")

    clip_model = clip_model.float()
    full_feats = clip_model.features_from_tokens(tokens.to(cuda_device, non_blocking=True), step_size=step)
    print(torch.sum(torch.abs(sdutils.norm_t(half_feats[0]) - sdutils.norm_t(full_feats[0]))))

test()

tensor(3.4897e-08, device='cuda:0')

In [8]:
features = torch.cat(feature_stack, axis=0)

In [10]:
del feature_stack

In [11]:
torch.save(features, features_path)

In [ ]:
feature_stack = []
step = 10000
with torch.no_grad():
    for start in range(0, len(old_toks), step):
        print(start)
        new_features = clip_model.get_features(old_toks[start:start+step].cuda())
        feature_stack.append(new_features.cpu())

    features = torch.stack(tuple(feature_stack))
    torch.save(features, features_path_old)